In [1]:
import pandas as pd
import requests
import json
import os
from groq import Groq

from pandasai import SmartDataframe

import os
from langchain_groq import ChatGroq
import semtui

semtui.SEMTUI_URI = "http://localhost:3003/api/"

zip_file_path = "C:\\Users\\user\\Documents\\BICOCCA\\archive (3).zip"
path_to_csv_file = "C:\\Users\\user\\Documents\\BICOCCA\SEMNOTE--\\SemTpy\\Museums.csv"
path_to_json_file = "C:\\Users\\user\\Downloads\\archive (4)\\continent.json"

In [2]:
# Example usage:
# Load data from a CSV file as a DataFrame
df_csv = semtui.load_local_data(path_to_csv_file)
print(df_csv.head())  # Display the first few rows of the DataFrame

                                 Point of Interest           Place  \
0  John F. Kennedy Presidential Library and Museum  Columbia Point   
1            Petrie Museum of Egyptian Archaeology          London   
2                             Helsinki City Museum     Kruununhaka   
3                                    Castle Caputh          Caputh   
4                                  Nyköping Castle        Nyköping   

            Adm1         Country Foundation date  \
0  Massachusetts   United States      1979-01-01   
1        England  United Kingdom      1892-01-01   
2        Uusimaa         Finland      1911-01-01   
3    Brandenburg         Germany      1662-01-01   
4   Södermanland          Sweden      1200-01-01   

                                               Owner  
0       National Archives and Records Administration  
1                          University College London  
2                                           Helsinki  
3  Prussian Palaces and Gardens Foundation Ber

In [3]:
df_museum = df_csv.head()
df_museum

,Point of Interest,Place,Adm1,Country,Foundation date,Owner
0,John F. Kennedy Presidential Library and Museum,Columbia Point,Massachusetts,United States,1979-01-01,National Archives and Records Administration
1,Petrie Museum of Egyptian Archaeology,London,England,United Kingdom,1892-01-01,University College London
2,Helsinki City Museum,Kruununhaka,Uusimaa,Finland,1911-01-01,Helsinki
3,Castle Caputh,Caputh,Brandenburg,Germany,1662-01-01,Prussian Palaces and Gardens Foundation Berlin...
4,Nyköping Castle,Nyköping,Södermanland,Sweden,1200-01-01,National Property Board


In [26]:
%pip install arrow

                                              0.0/66.4 kB ? eta -:--:--
     ------------------                       30.7/66.4 kB ? eta -:--:--
     ------------------                       30.7/66.4 kB ? eta -:--:--
     ------------------                       30.7/66.4 kB ? eta -:--:--
     -------------------------------------- 66.4/66.4 kB 357.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [31]:
import arrow
import pandas as pd

def convert_to_iso8601_with_arrow(df, date_col_name):
    # Temporarily disable the setting with copy warning
    pd.options.mode.chained_assignment = None  # default='warn'
    
    # Convert the date column to Arrow objects, then back to ISO 8601 formatted strings
    for index in df.index:
        original_date = df.loc[index, date_col_name]
        formatted_date = arrow.get(original_date).format('YYYY-MM-DD')
        df.loc[index, date_col_name] = formatted_date
    
    # Re-enable the setting with copy warning
    pd.options.mode.chained_assignment = 'warn'
    return df

In [32]:
# Convert the 'Foundation date' column using Arrow
df = convert_to_iso8601_with_arrow(df_museum, 'Foundation date')

# Display the updated DataFrame
print(df)

                                 Point of Interest           Place  \
0  John F. Kennedy Presidential Library and Museum  Columbia Point   
1            Petrie Museum of Egyptian Archaeology          London   
2                             Helsinki City Museum     Kruununhaka   
3                                    Castle Caputh          Caputh   
4                                  Nyköping Castle        Nyköping   

            Adm1         Country Foundation date  \
0  Massachusetts   United States      1979-01-01   
1        England  United Kingdom      1892-01-01   
2        Uusimaa         Finland      1911-01-01   
3    Brandenburg         Germany      1662-01-01   
4   Södermanland          Sweden      1200-01-01   

                                               Owner  
0       National Archives and Records Administration  
1                          University College London  
2                                           Helsinki  
3  Prussian Palaces and Gardens Foundation Ber

In [23]:
# Function to convert date column to ISO 8601 format using pandas
def convert_to_iso8601_pandas(df, date_col_name):
    # Check if the date column exists in the DataFrame
    if date_col_name not in df.columns:
        print(f"The specified column '{date_col_name}' does not exist in the DataFrame.")
        return df

    # Convert each date in the column to ISO 8601 format
    try:
        df[date_col_name] = pd.to_datetime(df[date_col_name]).dt.strftime('%Y-%m-%dT%H:%M:%S%z')
    except Exception as e:
        print(f"An error occurred while converting dates: {e}")
    return df



In [25]:
# Call the function and pass the DataFrame and the name of the date column
date_col_name = 'Foundation date'  # Replace with the actual name of your date column
df = convert_to_iso8601_pandas(df_museum, date_col_name)

# Display the updated DataFrame
df

An error occurred while converting dates: Out of bounds nanosecond timestamp: 1662-01-01 00:00:00 present at position 3


,Point of Interest,Place,Adm1,Country,Foundation date,Owner
0,John F. Kennedy Presidential Library and Museum,Columbia Point,Massachusetts,United States,1979-01-01,National Archives and Records Administration
1,Petrie Museum of Egyptian Archaeology,London,England,United Kingdom,1892-01-01,University College London
2,Helsinki City Museum,Kruununhaka,Uusimaa,Finland,1911-01-01,Helsinki
3,Castle Caputh,Caputh,Brandenburg,Germany,1662-01-01,Prussian Palaces and Gardens Foundation Berlin...
4,Nyköping Castle,Nyköping,Södermanland,Sweden,1200-01-01,National Property Board


In [ ]:
df_json = semtui.load_local_data(path_to_json_file)
print(df_json.head())  # Display the first few rows of the DataFrame

In [ ]:
# Load data from a ZIP file containing multiple CSV and JSON files as a list of DataFrames
#zip_file_path = "path/to/your/files.zip"
list_dfs = semtui.load_local_data(zip_file_path, file_type='zip', load_as='List')

# Iterate through the list and print the first few rows of each DataFrame
for df in list_dfs:
    print(df.head())


In [4]:
#%pip install ipyaggrid

semtui.display_csv_in_grid(path_to_csv_file)

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='buttons', height='350px', menu={'buttons': [{…

In [5]:
import os 
from dotenv import load_dotenv

load_dotenv(override=True)

#print(os.getenv("GROQ_API_KEY"))

True

In [ ]:
%pip install typing-inspect==0.8.0 typing_extensions==4.5.0

In [ ]:
%pip install pydantic==1.10.11

In [ ]:
%pip install -q -U langchain langchain_core langchain_groq

In [ ]:
import os
from langchain_groq import ChatGroq

#Groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model_name="mixtral-8x7b-32768", 
    api_key=os.getenv("GROQ_API_KEY")
)


In [ ]:
%pip install pandasai 

In [ ]:

df = SmartDataframe(df_csv, config={"llm": llm})

In [ ]:
df.chat('What are the features of this data?')

In [6]:
import os
# Assuming 'df_csv' is your DataFrame and 'my_api_key' is your API key
my_api_key= os.getenv("GROQ_API_KEY")
print(my_api_key)
recommendations = semtui.get_transformation_recommendations_from_df(df_csv, my_api_key)
print(recommendations)

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
Feature Names:
['Point of Interest', 'Place', 'Adm1', 'Country', 'Foundation date', 'Owner']

Sure, I can suggest some data transformations based on the data schema you provided. Here are some ideas that involve using external APIs or data sources:

1. Geocoding: You can use a geocoding API (such as the Google Maps Geocoding API or the OpenCage Geocoder) to transform the "Place" and "Adm1" fields into more specific geographic coordinates (latitude and longitude). This can be useful for mapping or spatial analysis.
2. Timezone lookup: You can use a timezone API (such as the Google Time Zone API or the tz-lookup library) to transform the "Foundation date" field into a specific timezone, based on the "Place" or "Adm1" fields. This can be useful for analyzing data across different timezones.
3. Currency conversion: If the "Owner" field represents a monetary value, you can use a currency conversion API (such as Open Exchange Rates or 

Sure, I can suggest some data transformations based on the data schema you provided. Here are some ideas that involve using external APIs or data sources:

1. Geocoding: You can use a geocoding API (such as the Google Maps Geocoding API or the OpenCage Geocoder) to transform the "Place" and "Adm1" fields into more specific geographic coordinates (latitude and longitude). This can be useful for mapping or spatial analysis.
2. Timezone lookup: You can use a timezone API (such as the Google Time Zone API or the tz-lookup library) to transform the "Foundation date" field into a specific timezone, based on the "Place" or "Adm1" fields. This can be useful for analyzing data across different timezones.
3. Currency conversion: If the "Owner" field represents a monetary value, you can use a currency conversion API (such as Open Exchange Rates or Currency Converter API) to transform the "Owner" field into a common currency, based on the "Country" field. This can be useful for comparing financial data across different countries.
4. Demographic data: You can use a demographic data API (such as the US Census Bureau API or the World Bank API) to enrich your data with demographic information, based on the "Country" or "Adm1" fields. This can be useful for analyzing data in the context of population trends or economic indicators.
5. Weather data: You can use a weather data API (such as OpenWeather or Weatherbit) to enrich your data with weather information, based on the "Place" or "Adm1" fields and the "Foundation date". This can be useful for analyzing data in the context of weather patterns or climate trends.NoneNone


In [ ]:

# Step 2: Extract the feature names from the DataFrame
feature_names = df_csv.columns.tolist()
print(f"Feature Names:\n{feature_names}\n")

# Format the feature names as a JSON object
feature_json = json.dumps({feature: 'float' for feature in feature_names})
try:

    client = Groq(
        api_key=os.getenv("GROQ_API_KEY"),
    )

    stream = client.chat.completions.create(
        #
        # Required parameters
        #
        messages=[
            # Set an optional system message. This sets the behavior of the
            # assistant and can be used to provide specific instructions for
            # how it should behave throughout the conversation.
            {
                "role": "system",
                "content": "you are a helpful assistant. Your job is to return recommendations for data transformations that involve external available on te internet or external api's."
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": f"{feature_json}",
            }
        ],

        # The language model which will generate the completion.
        model="mixtral-8x7b-32768",

        temperature=0.0,

        max_tokens=1024,

        top_p=1,

        stop=None,

        stream=True,
    )

    # Print the incremental deltas returned by the LLM.
    for chunk in stream:
        print(chunk.choices[0].delta.content, end="")

except requests.exceptions.RequestException as e:
    print("Error making the request:", e)
except json.JSONDecodeError as e:
    print("Error decoding JSON response:", e)
except Exception as e:
    print("An error occurred:", e)

In [ ]:

import pandas as pd

# Example usage:
# Assuming 'df_csv' is your DataFrame and you want to ask a question
df = pd.DataFrame({
    "country": [
        "United States",
        "United Kingdom",
        "France",
        "Germany",
        "Italy",
        "Spain",
        "Canada",
        "Australia",
        "Japan",
        "China",
    ],
    "gdp": [
        19294482071552,
        2891615567872,
        2411255037952,
        3435817336832,
        1745433788416,
        1181205135360,
        1607402389504,
        1490967855104,
        4380756541440,
        14631844184064,
    ],
    "happiness_index": [6.94, 7.16, 6.66, 7.07, 6.38, 6.4, 7.23, 7.22, 5.87, 5.12],
})

answer = semtui.df_chat(df, 'Return the top 5 countries by GDP?')
print(f"Answer:\n{answer}")

# Example usage:
# Assuming 'df_csv' is your DataFrame and you want to ask a question
# answer = df_chat(df_csv, 'What are the top 5 countries?')
# print(f"Answer:\n{answer}")

In [7]:
import requests
import pandas as pd

def get_geocoded_data(df, api_key):
    base_url = "https://geocode.search.hereapi.com/v1/geocode"
    resolved_addresses = []

    for index, row in df.iterrows():
        # Concatenate the address components
        address_query = f"{row['Point of Interest']}, {row['Place']}, {row['Adm1']}, {row['Country']}"
        params = {
            'q': address_query,
            'apiKey': api_key
        }

        # Construct the full URL for debugging purposes
        request_url = f"{base_url}?q={address_query}&apiKey={api_key}"
        #print(f"Request URL: {request_url}")

        # Make the GET request to the geocoding API
        response = requests.get(base_url, params=params)
        #print(f"Status Code: {response.status_code}")

        if response.status_code == 200:
            data = response.json()
            #print(f"Response Data: {data}")  # Print the entire response data for debugging
            items = data.get('items', [])
            
            if items:
                resolved_address = items[0]['address']['label']
                lat = items[0]['position']['lat']
                lng = items[0]['position']['lng']
                
                # Append the resolved data including the correct postal code and coordinates
                resolved_addresses.append({
                    'Original Address': address_query,
                    'Resolved Address': resolved_address,
                    'Latitude': lat,
                    'Longitude': lng
                })
            else:
                # In case no items were found
                resolved_addresses.append({
                    'Original Address': address_query,
                    'Resolved Address': None,
                    'Latitude': None,
                    'Longitude': None
                })
        else:
            print(f"Failed to geocode address: {address_query}. Status code: {response.status_code}")
            resolved_addresses.append({
                'Original Address': address_query,
                'Resolved Address': 'Error',
                'Latitude': 'Error',
                'Longitude': 'Error'
            })

    # Convert the resolved addresses to a DataFrame
    return pd.DataFrame(resolved_addresses)


# Assuming your DataFrame is named 'df' and you have an API key
api_key = '9HxvlAKEy7YhmiYLVcaNLxhYjuw1D4Emd05KnuOHrBE'
geocoded_df = get_geocoded_data(df_museum, api_key)
print(geocoded_df)


Request URL: https://geocode.search.hereapi.com/v1/geocode?q=John F. Kennedy Presidential Library and Museum, Columbia Point, Massachusetts, United States&apiKey=9HxvlAKEy7YhmiYLVcaNLxhYjuw1D4Emd05KnuOHrBE
Status Code: 200
Response Data: {'items': [{'title': 'John F. Kennedy Presidential Library and Museum', 'id': 'here:pds:place:840drt2x-04d51cc7b6744710a23823c5c9f64653', 'resultType': 'place', 'address': {'label': 'John F. Kennedy Presidential Library and Museum, Columbia Pt, Dorchester, MA 02125, United States', 'countryCode': 'USA', 'countryName': 'United States', 'stateCode': 'MA', 'state': 'Massachusetts', 'county': 'Suffolk', 'city': 'Dorchester', 'district': 'Columbia Point', 'street': 'Columbia Pt', 'postalCode': '02125'}, 'position': {'lat': 42.31597, 'lng': -71.03407}, 'access': [{'lat': 42.31597, 'lng': -71.03407}], 'categories': [{'id': '300-3100-0000', 'name': 'Museum', 'primary': True}], 'scoring': {'queryScore': 1.0, 'fieldScore': {'country': 1.0, 'state': 1.0, 'streets

In [8]:
geocoded_df

,Original Address,Resolved Address,Latitude,Longitude
0,John F. Kennedy Presidential Library and Museu...,John F. Kennedy Presidential Library and Museu...,42.31597,-71.03407
1,"Petrie Museum of Egyptian Archaeology, London,...","Ucl Art Museum, Gower Street, London, WC1E 6BT...",51.52471,-0.13351
2,"Helsinki City Museum, Kruununhaka, Uusimaa, Fi...","Helsingin kaupunginmuseo, Aleksanterinkatu 16,...",60.16862,24.95420
3,"Castle Caputh, Caputh, Brandenburg, Germany","Weg nach Caputh, 14552 Michendorf, Deutschland",52.32503,13.04364
4,"Nyköping Castle, Nyköping, Södermanland, Sweden","Nyköpingshus, Vallgatan 12, SE-611 32 Nyköping...",58.74937,17.00960


In [ ]:
import aiohttp
import asyncio
import time

async def test_api(api_url: str, params: dict) -> float:
    async with aiohttp.ClientSession() as session:
        start_time = time.time()
        async with session.get(api_url, params=params) as response:
            elapsed_time = time.time() - start_time
            return elapsed_time

async def pin_and_report(api_url: str, params: dict) -> None:
    print("Pinning the API to test availability...")
    start_time = time.time()
    try:
        response_time = await test_api(api_url, params)
        print(f"API is available. Response time: {response_time:.2f} seconds")
    except Exception as e:
        print(f"API is not available. Error: {str(e)}")
    finally:
        total_time = time.time() - start_time
        print(f"Total time taken: {total_time:.2f} seconds")



In [12]:
# Example usage:
api_url = "https://geocode.search.hereapi.com/v1/geocode"
params = {
    'q': "New York, NY, USA",  # A sample address for testing
    'apiKey': '9HxvlAKEy7YhmiYLVcaNLxhYjuw1D4Emd05KnuOHrBE'
}
await pin_and_report(api_url, params)

Pinning the API to test availability...
API is available. Response time: 0.28 seconds
API is available. Response time: 0.17 seconds
Total time taken for 2 records: 0.91 seconds


0.9110591411590576

In [ ]:
# Example usage:
api_url = "https://geocode.search.hereapi.com/v1/geocode"
params = {
    'q': "New York, NY, USA",  # A sample address for testing
    'apiKey': '9HxvlAKEy7YhmiYLVcaNLxhYjuw1D4Emd05KnuOHrBE'
}

# Convert DataFrame rows to dictionaries
unique_records = df_museum.to_dict(orient='records')

asyncio.run(pin_and_report(api_url, unique_records))

In [13]:
%pip install folium


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



                                              0.0/100.0 kB ? eta -:--:--
     -------------------------------------- 100.0/100.0 kB 6.0 MB/s eta 0:00:00
                                              0.0/56.3 kB ? eta -:--:--
     ---------------------------------------- 56.3/56.3 kB ? eta 0:00:00


In [14]:
import pandas as pd
import folium

def create_map(geocoded_df):
    # Create a map instance centered around a geographic midpoint or a specific location
    m = folium.Map(location=[geocoded_df['Latitude'].mean(), geocoded_df['Longitude'].mean()], zoom_start=2)

    # Add markers to the map for each geocoded address
    for idx, row in geocoded_df.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"{row['Original Address']}<br>Resolved: {row['Resolved Address']}",
            tooltip=row['Original Address']
        ).add_to(m)

    # Display the map
    return m


# Create the map using the function
map = create_map(geocoded_df)



In [15]:
# Display the map
map

Here are a few alternatives:

- Google Maps Street View API: While not strictly open source, Google Maps Street View API provides extensive street-level imagery. It allows you to embed interactive panoramas into your web or mobile applications.

- Leaflet: Leaflet is an open-source JavaScript library for interactive maps. It supports street view through plugins such as Leaflet Street View, which integrates Google Street View with Leaflet maps.

- OpenStreetMap (OSM): OpenStreetMap is a collaborative mapping project that provides free geographic data. There are various JavaScript libraries, such as OpenLayers and Mapbox GL JS, that you can use to integrate OSM data, including street-level views, into your applications.

- Mapbox: Mapbox offers a suite of mapping tools and APIs, including Mapbox Streets, which provides street-level mapping data. Mapbox GL JS is a powerful JavaScript library for creating interactive maps with Mapbox data.

- Here Maps API: Here Maps provides mapping and location-based services, including street-level views. You can use the Here Maps API to integrate street view functionality into your applications.

- Mapillary: Mapillary is a platform for street-level imagery and map data. It offers APIs and SDKs for developers to access and integrate street view data into their applications.


In [16]:
%pip install opencage


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)


In [17]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode

def get_geocoded_data_oc(df, api_key):
    geocoder = OpenCageGeocode(api_key)
    resolved_addresses = []

    for index, row in df.iterrows():
        # Concatenate the address components
        address_query = f"{row['Point of Interest']}, {row['Place']}, {row['Adm1']}, {row['Country']}"

        try:
            # Use the OpenCage Geocode API to fetch the geocode
            results = geocoder.geocode(address_query, no_annotations='1')

            if results and len(results):
                # Extract the first result's longitude and latitude
                longitude = results[0]['geometry']['lng']
                latitude  = results[0]['geometry']['lat']
                resolved_address = results[0]['formatted']  # Get the formatted address

                # Append the resolved data
                resolved_addresses.append({
                    'Original Address': address_query,
                    'Resolved Address': resolved_address,
                    'Latitude': latitude,
                    'Longitude': longitude
                })
            else:
                # In case no results were found
                resolved_addresses.append({
                    'Original Address': address_query,
                    'Resolved Address': None,
                    'Latitude': None,
                    'Longitude': None
                })
        except Exception as e:
            # Handle any errors that occur during the geocoding process
            print(f"Error geocoding address: {address_query}. Error message: {e}")
            resolved_addresses.append({
                'Original Address': address_query,
                'Resolved Address': 'Error',
                'Latitude': 'Error',
                'Longitude': 'Error'
            })

    # Convert the resolved addresses to a DataFrame
    return pd.DataFrame(resolved_addresses)


In [18]:

api_key = 'b8f07131fa4643fbb513b359d26ee51c'

# Assuming the get_geocoded_data_oc function is already defined as shown earlier
geocoded_df = get_geocoded_data_oc(df_museum, api_key)

print(geocoded_df)


                                    Original Address  \
0  John F. Kennedy Presidential Library and Museu...   
1  Petrie Museum of Egyptian Archaeology, London,...   
2  Helsinki City Museum, Kruununhaka, Uusimaa, Fi...   
3        Castle Caputh, Caputh, Brandenburg, Germany   
4    Nyköping Castle, Nyköping, Södermanland, Sweden   

                                    Resolved Address   Latitude  Longitude  
0  Suffolk County, Massachusetts, United States o...  42.316210 -71.042550  
1                             London, United Kingdom  51.508530  -0.125740  
2  Helsinki City Museum, Aleksanterinkatu 16, 001...  60.168754  24.953917  
3                       Caputh, Brandenburg, Germany  52.346980  12.992210  
4                 Nyköping, Nyköpings Kommun, Sweden  58.753000  17.007880  


In [19]:
geocoded_df

,Original Address,Resolved Address,Latitude,Longitude
0,John F. Kennedy Presidential Library and Museu...,"Suffolk County, Massachusetts, United States o...",42.316210,-71.042550
1,"Petrie Museum of Egyptian Archaeology, London,...","London, United Kingdom",51.508530,-0.125740
2,"Helsinki City Museum, Kruununhaka, Uusimaa, Fi...","Helsinki City Museum, Aleksanterinkatu 16, 001...",60.168754,24.953917
3,"Castle Caputh, Caputh, Brandenburg, Germany","Caputh, Brandenburg, Germany",52.346980,12.992210
4,"Nyköping Castle, Nyköping, Södermanland, Sweden","Nyköping, Nyköpings Kommun, Sweden",58.753000,17.007880


In [20]:
import requests
import pandas as pd

def add_yearly_average_temperature(df, api_key):
    url_template = "https://history.openweathermap.org/data/2.5/aggregated/year?lat={latitude}&lon={longitude}&appid={api_key}"

    # Prepare a list to hold temperature data
    average_temperatures_data = []

    for index, row in df.iterrows():
        latitude = row['Latitude']
        longitude = row['Longitude']
        
        # Skip rows where latitude or longitude is None
        if pd.isnull(latitude) or pd.isnull(longitude):
            average_temperatures_data.append((None, None))
            continue

        url = url_template.format(latitude=latitude, longitude=longitude, api_key=api_key)

        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for HTTP errors
            data = response.json()

            if 'result' in data:
                result = data['result']
                # Assuming you want the yearly average temperature
                yearly_average_temperature = sum([entry['temp']['mean'] for entry in result]) / len(result)
                average_temperatures_data.append(yearly_average_temperature)
            else:
                average_temperatures_data.append(None)

        except requests.RequestException as e:
            print(f"Error fetching temperature data for {latitude}, {longitude}: {e}")
            average_temperatures_data.append(None)

    # Add the average temperatures data to the DataFrame
    df['Yearly Average Temperature (K)'] = average_temperatures_data
    return df


In [22]:

api_key = '961b2326f0ecd7a40bddfd04c3b3d28d'
geocoded_df_with_temps = add_yearly_average_temperature(geocoded_df, api_key)

print(geocoded_df_with_temps)



Error fetching temperature data for 42.31621, -71.04255: 401 Client Error: Unauthorized for url: https://history.openweathermap.org/data/2.5/aggregated/year?lat=42.31621&lon=-71.04255&appid=961b2326f0ecd7a40bddfd04c3b3d28d
Error fetching temperature data for 51.50853, -0.12574: 401 Client Error: Unauthorized for url: https://history.openweathermap.org/data/2.5/aggregated/year?lat=51.50853&lon=-0.12574&appid=961b2326f0ecd7a40bddfd04c3b3d28d
Error fetching temperature data for 60.1687535, 24.9539174: 401 Client Error: Unauthorized for url: https://history.openweathermap.org/data/2.5/aggregated/year?lat=60.1687535&lon=24.9539174&appid=961b2326f0ecd7a40bddfd04c3b3d28d
Error fetching temperature data for 52.34698, 12.99221: 401 Client Error: Unauthorized for url: https://history.openweathermap.org/data/2.5/aggregated/year?lat=52.34698&lon=12.99221&appid=961b2326f0ecd7a40bddfd04c3b3d28d
Error fetching temperature data for 58.753, 17.00788: 401 Client Error: Unauthorized for url: https://hist

In [ ]:
#print(semtui.addDataset(zip_file_path, 'Adventure_Dataset', inputType='file'))


In [ ]:
# Check the tables in the uploaded dataset
#semtui.checkDataset(zip_file_path)

In [ ]:
#tables = semtui.load_tables(zip_file_path)

# Access a specific table DataFrame
# If using the full filename as the key
#df_product = tables.get('Product.csv')
# Or if using the base name as the key (assuming you made the suggested modification)
#df_product = tables.get('Product')

In [ ]:
#datasets = semtui.getDatasetsList()
#print(datasets)

In [ ]:
semtui.getExtendersList()

In [ ]:
#semtui.reconcile(df_csv, df_csv['Place'], 1)